In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, Lambda, Dropout, Embedding, Multiply
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime
import pickle

def r_squared(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - ss_res / (ss_tot + tf.keras.backend.epsilon())

class MoEWeightingLayer(tf.keras.layers.Layer):
    def __init__(self, num_experts, **kwargs):
        super(MoEWeightingLayer, self).__init__(**kwargs)
        self.num_experts = num_experts

    def build(self, input_shape):
        self.w = self.add_weight(name='weights', shape=(input_shape[1], self.num_experts), initializer='uniform', trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w)

def build_moe_model(num_offices, num_species, input_shape, expert_configs, dropout_rate=0.1):
    main_input = Input(shape=input_shape)
    office_input = Input(shape=(1,))
    species_input = Input(shape=(1,))
    
    expert_outputs = []
    
    # Iterate over each expert's configuration and log the configuration
    for idx, config in enumerate(expert_configs):
        print(f"Building expert {idx+1} with {config['layers']} layers and {config['neurons']} neurons per layer")
        x = main_input
        for i in range(config['layers']):
            return_sequences = i < (config['layers'] - 1)  # Only last layer does not return sequences
            x = LSTM(config['neurons'], return_sequences=return_sequences)(x)
            x = Dropout(dropout_rate)(x)
        expert_outputs.append(Dense(1, activation='linear')(x))  # Linear activation for continuous output

    concatenated = Concatenate()(expert_outputs)
    office_weights_output = Dense(num_offices, activation='softmax')(concatenated)
    office_output = Lambda(lambda x: tf.reduce_sum(x, axis=1))(office_weights_output)

    species_embedding = Embedding(num_species, 1, input_length=1)(species_input)
    species_embedding = Lambda(lambda x: tf.squeeze(x, axis=-1))(species_embedding)
    adjusted_output = Multiply()([office_output, species_embedding])

    model = Model(inputs=[main_input, office_input, species_input], outputs=adjusted_output)
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy',r_squared, tf.keras.metrics.RootMeanSquaredError()])
    return model

def prepare_data_and_train_model(filename,expert_configs):
    df = pd.read_csv(filename)
    office_encoder = LabelEncoder()
    species_encoder = LabelEncoder()
    df['NOMBRE_OFICINA_encoded'] = office_encoder.fit_transform(df['NOMBRE OFICINA'])
    df['NOMBRE_PRINCIPAL_encoded'] = species_encoder.fit_transform(df['NOMBRE PRINCIPAL'])
    
    scaler_sst = MinMaxScaler()
    scaler_weight = MinMaxScaler()
    df['SST_scaled'] = scaler_sst.fit_transform(df[['SST']])
    df['PESO_DESEMBARCADO_scaled'] = scaler_weight.fit_transform(df[['PESO DESEMBARCADO_KILOGRAMOS']])
    
    X = np.array(df[['SST_scaled']])
    y = np.array(df['PESO_DESEMBARCADO_scaled'])
    offices = np.array(df['NOMBRE_OFICINA_encoded'])
    species = np.array(df['NOMBRE_PRINCIPAL_encoded'])

    X_train, X_test, y_train, y_test, offices_train, offices_test, species_train, species_test = train_test_split(
        X, y, offices, species, test_size=0.2, random_state=42)

    num_offices = df['NOMBRE_OFICINA_encoded'].nunique()
    num_species = df['NOMBRE_PRINCIPAL_encoded'].nunique()
    
    model = build_moe_model(num_offices, num_species, (X_train.shape[1], 1),expert_configs)
    
    log_dir = "logs/fit/" + "MoE"+datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    early_stop = EarlyStopping(monitor='val_loss', patience=25, mode='min', restore_best_weights=True)
    model.fit([X_train, offices_train, species_train], y_train, epochs=500, batch_size=128, validation_split=0.2,
              callbacks=[early_stop, tensorboard_callback], verbose=1)

    mse = model.evaluate([X_test, offices_test, species_test], y_test, verbose=0)
    print(f'Test MSE: {mse}')
    
    return model, office_encoder, species_encoder, scaler_sst, scaler_weight


# Example of how to use the returned objects

expert_configs = [{'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3},
                  {'neurons': 15, 'layers': 3}]

model, office_encoder, species_encoder, scaler_sst, scaler_weight = prepare_data_and_train_model('aggregated_data4.csv',expert_configs)



2024-05-04 08:44:43.458135: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-04 08:44:43.461449: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 08:44:43.493652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 08:44:43.493671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 08:44:43.494534: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Building expert 1 with 3 layers and 15 neurons per layer
Building expert 2 with 3 layers and 15 neurons per layer
Building expert 3 with 3 layers and 15 neurons per layer
Building expert 4 with 3 layers and 15 neurons per layer
Building expert 5 with 3 layers and 15 neurons per layer
Building expert 6 with 3 layers and 15 neurons per layer
Building expert 7 with 3 layers and 15 neurons per layer
Building expert 8 with 3 layers and 15 neurons per layer
Building expert 9 with 3 layers and 15 neurons per layer
Building expert 10 with 3 layers and 15 neurons per layer
Building expert 11 with 3 layers and 15 neurons per layer
Building expert 12 with 3 layers and 15 neurons per layer
Building expert 13 with 3 layers and 15 neurons per layer
Building expert 14 with 3 layers and 15 neurons per layer
Building expert 15 with 3 layers and 15 neurons per layer
Epoch 1/500
1334/1334 [==============================] - 64s 17ms/step - loss: 3.6561e-05 - accuracy: 2.9287e-05 - r_squared: -4.7745 - roo

In [ ]:
# Saving the encoders and scaler
with open('office_encoder.pkl', 'wb') as file:
    pickle.dump(office_encoder, file)
with open('species_encoder.pkl', 'wb') as file:
    pickle.dump(species_encoder, file)
with open('sst_scaler.pkl', 'wb') as file:
    pickle.dump(scaler_sst, file)
with open('weight_scaler.pkl', 'wb') as file:
    pickle.dump(scaler_weight, file)

Building expert 1 with 3 layers and 75 neurons per layer
Epoch 1/50
2083/5336 [==========>...................] - ETA: 10s - loss: 3.4276e-05 - mse: 3.4276e-05 - r_squared: -37.3426 - root_mean_squared_error: 0.0059

KeyboardInterrupt: 

In [ ]:
import pickle

# Function to prepare new data using the loaded encoders and scaler
def prepare_new_data(new_data, office_encoder, species_encoder, scaler):
    # Assume new_data is a DataFrame with columns 'SST', 'NOMBRE_OFICINA', 'NOMBRE_PRINCIPAL'
    new_data['NOMBRE_OFICINA_encoded'] = office_encoder.transform(new_data['NOMBRE_OFICINA'])
    new_data['NOMBRE_PRINCIPAL_encoded'] = species_encoder.transform(new_data['NOMBRE_PRINCIPAL'])
    
    # Applying the MinMax scaling to the SST column
    new_data['SST_scaled'] = scaler.transform(new_data[['SST']])
    
    X = np.array(new_data['SST_scaled'])
    offices = np.array(new_data['NOMBRE_OFICINA_encoded'])
    species = np.array(new_data['NOMBRE_PRINCIPAL_encoded'])
    
    # Assuming that the input_shape[1] (feature dimension) is 1
    input_shape = (X.shape[1], 1)
    return [X.reshape(-1, input_shape[0], input_shape[1]), offices, species]

# Example usage:
# new_data = pd.DataFrame({
#     'SST': [25.5],  # Example sea surface temperature
#     'NOMBRE_OFICINA': ['Office1'],  # Example office name
#     'NOMBRE_PRINCIPAL': ['Species1']  # Example species name
# })

# Load the scaler from disk
with open('sst_scaler.pkl', 'rb') as file:
    sst_scaler = pickle.load(file)

# Prepare the new data using the loaded encoders and scaler
prepared_data = prepare_new_data(new_data, office_encoder, species_encoder, sst_scaler)


In [ ]:
# Example function to use the model to predict with new data
def predict_with_new_data(model, prepared_data):
    predictions = model.predict(prepared_data)
    return predictions

# Example usage
new_data = pd.DataFrame({
    'SST': [25.5],  # Example sea surface temperature
    'NOMBRE_OFICINA': ['Office1'],  # Example office name
    'NOMBRE_PRINCIPAL': ['Species1']  # Example species name
})

prepared_data = prepare_new_data(new_data, office_encoder, species_encoder)
predictions = predict_with_new_data(model, prepared_data)
print("Predicted Catch:", predictions)